# Préparation des données

In [2]:
%run '2 - Récupération.ipynb'

## 1 - Inventaire des données

Après la réunion avec Selim, il nous a donné une marche à suivre très précise qui est détaillé dans le fichier [Rapport](../rapports/Rapport_2411.md). La première étape est de faire un inventaire des variables et de créer une base de donnée les contenant tous. Il faut bien garder en tête que les différents modèles ne verront que ligne par ligne.


### Variables qualitatives

   Nom de la variable    | Abréviation
   ----------------------|:-----------:
   Identifiant du joueur |  PLAYER_ID
   Équipe du joueur      |    TEAM
   Équipe adverse        |     ADV
   Domicile / Exterieur  |    HOME
   Adversaire directe    |   ADV_ID
 
### Variables quantitiatives

#### Variables scores positives

   Nom de la variable               | Abréviation 
   ---------------------------------|:-----------:
   Nombre de points                 |     PTS    
   Nombre de rebonds                |     REB
   Nombre de passes décisives       |     AST
   Nombre de récupération de balle  |     STL
   Nombre de block                  |     BLK
   Nombre de shoot réussi           |     FGM
   Nombre de lancer franc réussi    |     FTM
   Nombre de tire à 3 points réussi |     FG3M

#### Variables scores négatives

   Nom de la variable                | Abréviation 
   ----------------------------------|:-----------:
   Nombre de perte de balle          |     TOV
   Nombre de tire raté               |     FGF
   Nombre de lancer franc raté       |     FTF
   Nombre de tire à 3 points de raté |     FG3F

#### Variables de scores globales

   Nom de la variable            | Abréviation
   ------------------------------|:-----------:
   Somme des variables positives |    BONUS
   Somme des variables négatives |    MALUS
   Score du joueur               |    SCORE   
   
#### Variables laggées   
   
Toutes les variables ci-dessus ne sont pas accéssible pour le temps $t$ donc nous utilisons ces variables :
   
   Nom de la variable                                  |      Abréviation
   ----------------------------------------------------|:--------------------:
   Moyenne des N derniers matchs                       |   AVG_[ABREVIATION]
   Moyenne des N derniers matchs contre l'équipe [ADV] | AVG_ADV\\_[ABREVIATION]
   La série de victoire                                |        NB_VIC 
   La série de victoire contre l'équipe [ADV]          |       NB_VIC\\_ADV
   
### Variables temporelles

   Nom de la variable | Abréviation
   -------------------|:-----------:
   Date du match      |  GAME_DATE
   
### Hyperparamètres

   Nom du paramètre                                                     | Abréviation
   ---------------------------------------------------------------------|:-----------:
   Le nombre de match que l'on retient pour les moyennes                |      N
   Le nombre de match que l'on retient pour les moyennes par adversaire |    N_ADD


In [3]:
SEASON = 2019

# Hyperparamètre : le nombre de match que l'on regarde en arrière
N = 3
N_ADD = 1

# On récupère les données que l'on a récupéré précédemment
df = score_df(pd.read_csv(f'../data/boxscore_{SEASON}.csv'))

In [4]:
# La liste des variables dont on va récupérer les moyennes
list_var_avg = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FGM', 'FTM', 'FG3M',
                 'TOV', 'FGF', 'FG3F', 'FTF']

In [5]:
def avg_df(df, n=N):
    '''Calcul les moyennes en utilisant une 'rolling window' sur les variables 
    intéressantes
    
    Args :
        df : database avec les variables dans list_var_avg
        
    '''
    
    def avg(df):
        return df[list_var_avg].shift(periods=1)  \
                                .rolling(n)  \
                                .mean()  \
                                .add_prefix('AVG_')
                                
    
    # Calcul des moyennes pour chaque joueur puis on recréer une base
    df_mean = df.groupby('PLAYER_ID').apply(avg)  \
                                        .reset_index()  \
                                        .set_index('level_1')  \

    return pd.concat([df, df_mean.drop('PLAYER_ID', axis=1)], axis=1)

In [6]:
avg_df(df).sample(3)

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,TEAM,ADV,HOME,WIN,PTS,REB,AST,STL,BLK,FGM,FTM,FG3M,TOV,FGF,FG3F,FTF,BONUS,MALUS,SCORE,AVG_PTS,AVG_REB,AVG_AST,AVG_STL,AVG_BLK,AVG_FGM,AVG_FTM,AVG_FG3M,AVG_TOV,AVG_FGF,AVG_FG3F,AVG_FTF
17127,22019,1628405,21900499,2019-12-31,LAC,SAC,False,True,2,0,0,0,0,0,2,0,0,0,0,0,4,0,4,0.000000,0.333333,0.333333,1.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.666667,0.0,0.0
15610,22019,1628368,21900612,2020-01-15,SAC,DAL,True,False,27,7,12,1,0,11,3,2,3,6,2,6,63,17,46,25.666667,7.000000,8.666667,2.000000,1.333333,10.000000,4.333333,1.333333,4.666667,10.000000,3.0,0.0
10247,22019,203944,21900764,2020-02-06,NYK,ORL,True,True,22,8,4,1,2,10,2,0,2,6,1,0,49,9,40,17.333333,13.333333,4.000000,1.333333,0.000000,6.333333,4.666667,0.000000,3.000000,8.333333,3.0,2.0


In [7]:
def avg_add_df(df, n=N_ADD):
    # Calcul les moyennes en utilisant une 'rolling window' sur les variables 
    # intéressantes
    def avg_add(df):
        return df[list_var_avg].shift(periods=1)  \
                                .rolling(n)  \
                                .mean()  \
                                .add_prefix('AVG_ADD_')
    
    # Calcul des moyennes pour chaque joueur et pour chaque équipe 
    # puis on recréer une base
    df_add_mean = df.groupby('PLAYER_ID').apply(lambda x:
                                                x.groupby('ADV').apply(avg_add))  \
                                            .reset_index()  \
                                            .set_index('level_2')  \
    
    return pd.concat([df, df_add_mean.drop(['PLAYER_ID', 'ADV'], axis=1)]  \
                                                                      , axis=1)

In [8]:
# TRÈS LONG !!!
#avg_add_df(df).sample(3)

In [9]:
def nb_vic_df(df):
    
    # Code non vectoriel : 
    def nb_vic(df):
        # On calcul la serie de victoire jusqu'à une défaite
        serie = [0]
        for i in range(1, len(df)):
            serie += [serie[-1] + 1] if df.WIN.iloc[i-1] else [0]

        return pd.Series(serie, index=df.index, name='NB_VIC').to_frame()

    # On calcul les séries pour chaque joueur
    df_nb_vic = df.groupby('PLAYER_ID').apply(nb_vic)

    # On rajoute les séries à la fin de notre dataframe
    return pd.concat([df, df_nb_vic], axis=1)

In [10]:
nb_vic_df(df).sample(3)

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,TEAM,ADV,HOME,WIN,PTS,REB,AST,STL,BLK,FGM,FTM,FG3M,TOV,FGF,FG3F,FTF,BONUS,MALUS,SCORE,NB_VIC
12347,22019,1626188,21900174,2019-11-15,LAL,SAC,True,True,2,2,0,0,0,1,0,0,0,1,0,0,5,1,4,2
17573,22019,1628422,21900614,2020-01-16,NYK,PHX,True,False,6,2,0,0,0,3,0,0,2,3,2,0,11,7,4,0
18294,22019,1628969,21900426,2019-12-20,PHX,OKC,False,False,2,2,0,0,1,1,0,0,1,3,1,0,6,5,1,0


In [11]:
def nb_vic_add_df(df):
    
    # Code non vectoriel : 
    def nb_vic_add(df):

        # On calcul la serie de victoire jusqu'à une défaite
        serie = [0]
        for i in range(1, len(df)):
            serie += [serie[-1] + 1] if df.WIN.iloc[i-1] else [0]
        return pd.Series(serie, index=df.index, name='NB_VIC_ADD').to_frame()

    # On calcul les séries pour chaque joueur et pour chaque équipe
    df_nb_vic_add = df.groupby('PLAYER_ID').apply(lambda df: 
                        df.groupby('ADV').apply(nb_vic_add))  \
                                            .reset_index()  \
                                            .set_index('level_1')  \
                                            .drop('PLAYER_ID', axis=1)

    return pd.concat([df, df_nb_vic_add], axis=1)

In [12]:
# TRÈS LONG !!
#nb_vic_add_df(df)

In [13]:
# TRÈS LONG !!!
# La dataframe finale  

#df = nb_vic_add_df(nb_vic_df(avg_add_df(avg_df(df))))

#df.to_csv('../data/boxscore_2019_final.csv', index=False)